Batch execution of graphs
=====================
So far, we have learnt together how to create and change graph with DGL, and how to trigger computation on nodes and edges of a graph, which is useful to implement models that focus on representation learning on one single large graph like citation graph or knowledge graph.

However, there are also scenarios like learning sentence syntax trees ([Tai et al., 2015](https://arxiv.org/abs/1503.00075)) or chemical structures ([Jin et at., 2018](https://arxiv.org/abs/1802.04364)) where the goal is to learn representation to classify or extract features for a lot of individual graphs. 

In such applications, ability to batch computation on multiple graphs matters. And in this tutorial, we will demonstrate how DGL address batching graph with its BatchedGraph API.

In [ ]:
# A bit of setup, just ignore this cell
import matplotlib.pyplot as plt

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['animation.html'] = 'html5'

### Simple Graph Classification Task

To make it more concrete, let's use graph classification as our example. Graph classification is an important problem with applications across many fields – bioinformatics, cheminformatics, social network analysis, urban computing and cyber-security. Applying graph neural networks to this problem has been a popular approach recently ([Ying et al., 2018](https://arxiv.org/abs/1806.08804), [Cangea et al., 2018](https://arxiv.org/abs/1811.01287), [Knyazev et al., 2018](https://arxiv.org/abs/1811.09595), [Bianchi et al., 2019](https://arxiv.org/abs/1901.01343), [Liao et al., 2019](https://arxiv.org/abs/1901.01484), [Gao et al., 2019](https://openreview.net/forum?id=HJePRoAct7)).


In this tutorial, we will use a simple graph classification task. We create a synthetic dataset data.MiniGCDataset, which has 8 different types of graphs and each class has the same number of graph samples. And the task is decide which of the 8 types below each graph belongs to.

![](https://s3.us-east-2.amazonaws.com/dgl.ai/tutorial/batch/dataset_overview.png)


In [ ]:
from dgl.data import MiniGCDataset
import matplotlib.pyplot as plt
import networkx as nx
# A dataset with 80 samples, each graph is
# of size [10, 20]
dataset = MiniGCDataset(80, 10, 20)
graph, label = dataset[0]
fig, ax = plt.subplots()
nx.draw(graph.to_networkx())
ax.set_title('Class: {:d}'.format(label))
plt.show()

### Form a graph mini-batch
To train neural networks more efficiently, we need to **batch** multiple samples together to form a mini-batch. Batching fixed-shaped tensor inputs is quite easy (for example, batching two images of size 28×28 gives a tensor of shape 2×28×28). By contrast, batching graph inputs has two challenges:

- Graphs are sparse.
- Graphs can have various length (e.g. number of nodes and edges).

To address this, DGL provides a `dgl.batch()` API. It leverages the trick that a batch of graphs can be viewed as a large graph that have many disjoint connected components. Below is an illustration:

![](https://s3.us-east-2.amazonaws.com/dgl.ai/tutorial/batch/batch.png)

We define the following `collate` function to form a mini-batch from a given list of graph and label pairs.

In [ ]:
import dgl

def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)

The return type of `dgl.batch()` is still a graph (similar to the fact that a batch of tensors is still a tensor). This means that any code that works for one graph immediately works for a batch of graphs. More importantly, since DGL processes messages on all nodes and edges in parallel, this greatly improves efficiency.

### Graph Classifier
The graph classification can be proceeded as follows:
![](https://s3.us-east-2.amazonaws.com/dgl.ai/tutorial/batch/graph_classifier.png)
From a batch of graphs, we first perform message passing/graph convolution for nodes to “communicate” with others. After message passing, we compute a tensor for graph representation from node (and edge) attributes. This step may be called “readout/aggregation” interchangeably. Finally, the graph representations can be fed into a classifier g to predict the graph labels.

### Graph Convolution

Our graph convolution operation is basically the same as that for GCN (checkout our tutorial). The only difference is that we use a simpler normalization factor for aggregating messages: $c_{ij}=|N(v_i)|$. Therefore, the update equation becomes:
$$h^{(l+1)}_{v_i}=ReLU(b^{(l)}+\frac{1}{|N(v_i)|}\sum\limits_{v_j\in N(v_i)}h^{(l)}_{v_j}W^{(l)})$$
The replacement of summation by average is to balance nodes with different degrees, which gives a better performance for this experiment.

Note that the self edges added in the dataset initialization allows us to include the original node feature $h^{(l)}_v$.


In [ ]:
import dgl.function as fn
import torch
import torch.nn as nn


# Sends a message of node feature h.
msg = fn.copy_src(src='h', out='m')

def reduce(nodes):
    """Take an average over all neighbor node features hu and use it to
    overwrite the original node feature."""
    accum = torch.mean(nodes.mailbox['m'], 1)
    return {'h': accum}

class NodeApplyModule(nn.Module):
    """Update the node feature hv with ReLU(Whv+b)."""
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        # Initialize the node features with h.
        g.ndata['h'] = feature
        g.update_all(msg, reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

### Readout and Classification

For this demonstration, we consider initial node features to be their degrees. After two rounds of graph convolution, we perform a graph readout by averaging over all node features for each graph in the batch

$$h_g=\frac{1}{|V|}\sum\limits_{v\in V}h_v$$

In DGL, `dgl.mean_nodes()` handles this task for a batch of graphs with variable size. We then feed our graph representations into a classifier with one linear layer to obtain pre-softmax logits.


In [ ]:
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()

        self.layers = nn.ModuleList([
            GCN(in_dim, hidden_dim, F.relu),
            GCN(hidden_dim, hidden_dim, F.relu)])
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g):
        # For undirected graphs, in_degree is the same as
        # out_degree.
        h = g.in_degrees().view(-1, 1).float()
        for conv in self.layers:
            h = conv(g, h)
        g.ndata['h'] = h
        hg = dgl.mean_nodes(g, 'h')
        return self.classify(hg)

### Setup and Training
We create a synthetic dataset of 400 graphs with 10 ~ 20 nodes. 320 graphs constitute a training set and 80 graphs constitute a test set.


In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader

# Create training and test sets.
trainset = MiniGCDataset(320, 10, 20)
testset = MiniGCDataset(80, 10, 20)
# Use PyTorch's DataLoader and the collate function defined before.
data_loader = DataLoader(trainset, batch_size=32, shuffle=True,
                         collate_fn=collate)

# Create model
model = Classifier(1, 256, trainset.num_classes)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()

epoch_losses = []
for epoch in range(80):
    epoch_loss = 0
    for iter, (bg, label) in enumerate(data_loader):
        prediction = model(bg)
        loss = loss_func(prediction, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().item()
    epoch_loss /= (iter + 1)
    print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss))
    epoch_losses.append(epoch_loss)

The learning curve of a run is presented below:

In [ ]:
plt.title('cross entropy averaged over minibatches')
plt.plot(epoch_losses)
plt.show()

In [ ]:
model.eval()
# Convert a list of tuples to two lists
test_X, test_Y = map(list, zip(*testset))
test_bg = dgl.batch(test_X)
test_Y = torch.tensor(test_Y).float().view(-1, 1)
probs_Y = torch.softmax(model(test_bg), 1)
sampled_Y = torch.multinomial(probs_Y, 1)
argmax_Y = torch.max(probs_Y, 1)[1].view(-1, 1)
print('Accuracy of sampled predictions on the test set: {:.4f}%'.format(
    (test_Y == sampled_Y.float()).sum().item() / len(test_Y) * 100))
print('Accuracy of argmax predictions on the test set: {:4f}%'.format(
    (test_Y == argmax_Y.float()).sum().item() / len(test_Y) * 100))